In [ ]:
import visa
import time
import numpy as np


In [3]:
rm = visa.ResourceManager()
rm.list_resources()

('ASRL3::INSTR',
 'ASRL10::INSTR',
 'GPIB0::3::INSTR',
 'GPIB0::7::INSTR',
 'GPIB0::12::INSTR',
 'GPIB0::14::INSTR',
 'GPIB0::24::INSTR')

In [32]:
# simple.py

import wx

app = wx.App()

frame = wx.Frame(None, title='Simple application')
frame.Show()

app.MainLoop()

0

In [5]:
vsm = rm.open_resource('GPIB0::9::INSTR')

In [4]:
ametek7270 = rm.open_resource('USB0::0x0A2D::0x001B::09140123::RAW')
keithley6221 = rm.open_resource('GPIB0::14::INSTR')
guass = rm.open_resource('GPIB0::13::INSTR')

In [5]:
def gauss_field():
    f = guass.query("RDGFIELD?")
    f = round(float(f[:-2]) / 100) * 100
    return f

In [7]:
gauss_field()

1000

In [23]:
def next_stable_field(last_field=None, wait_count= 8, time_per_poind=0.5, max_wait=5*60):
    next_fields = []
    for i in range(int(max_wait/time_per_poind)):
        time.sleep(time_per_poind)
        current_field = gauss_field()
        if current_field == last_field:
            continue
        else:
            next_fields.append(current_field)
            if next_fields[-wait_count:] == [next_fields[-1]] * wait_count:
                return next_fields[-1]
    print("wait field time out")

In [13]:
print(next_stable_field())

[1000]
1


[1000, 1000]
2


[1000, 1000, 1000]
3


[1000, 1000, 1000, 1000]
4


[1000, 1000, 1000, 1000, 1000]
5


[1000, 1000, 1000, 1000, 1000, 1000]
6


[1000, 1000, 1000, 1000, 1000, 1000, 1000]
7


[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
8
1000


In [61]:
guass.query("*IDN?")

'LSCI,MODEL455,LSA12A6,11262013\r\n'

In [8]:
def num_to_hex(x):
    if x == 0:
        return "00000000"
    res = []
    if x > 0:
        res.append(str(0))
    else:
        res.append(str(8))
    x = abs(x)
    e = 0
    m = x
    while m > 2:
        m = m / 2
        e += 1

    res.append(hex(e+1)[2:])
    
    m = m * 128
    m_int = int(m)
    m_digit = m - m_int
    m_digit = int(m_digit * 65536)
    
    m_int = hex(m_int)[2:]
    m_int = "0" * (2-len(m_int)) + m_int
    m_digit = hex(m_digit)[2:]
    m_digit = "0" * (4-len(m_digit)) + m_digit
    
    res.append(m_int)
    res.append(m_digit)
    res = "".join(res).upper()
    assert (len(res)) == 8
    return res

In [9]:
def hex_to_num(s):
    sign = s[0]
    if sign == '0':
        sign = 1
    elif sign=='8':
        sign = -1
    else:
        return 0
        
    e = int(s[1],16) - 1
    m_int = s[2:4].lstrip('0')
    if len(m_int) == 0:
        m_int = '0'
    m_int = int(m_int,16)
    m_float = s[4:].lstrip('0')
    if len(m_float) == 0:
        m_float = '0'
    m_float = int(m_float,16)
    m = (m_int + (m_float / 65536))/128
    res = sign * m * 2** e
    return res

In [10]:
num_to_hex(199.989)

'08C7FD2F'

In [11]:
hex_to_num('08C7FD2F')

199.98899841308594

In [183]:
check = [0, 1, 10, 100, 1000, 10000, 888, -10, -100, -1000, -10000, -888]
for i in check:
    print( "{} convert to {}".format(i, num_to_hex(i)))

0 convert to 00000000
1 convert to 01800000
10 convert to 04A00000
100 convert to 07C80000
1000 convert to 0AFA0000
10000 convert to 0E9C4000
888 convert to 0ADE0000
-10 convert to 84A00000
-100 convert to 87C80000
-1000 convert to 8AFA0000
-10000 convert to 8E9C4000
-888 convert to 8ADE0000


In [39]:
def set_field(f, rate):
    if abs(f) > 15000:
        print("set field too high")
        raise
    vsm.write("CONTR " + num_to_hex(rate))
    vsm.write("CONTO " + num_to_hex(f))
    check_stable()
def read_field():
    f = vsm.query("READ?")[18:26].lower()
    if not(len(f) == 8):
        raise
    f = hex_to_num(f)
    return f

In [26]:
vsm.query("CONST?")

'2\r\n'

In [30]:
def check_stable(time_per_point=0.5):
    current = [0, 0 ,0]
    target = [2, 2, 2]
    while True:
        next_point = int(vsm.query("CONST?")[0])
        current.pop(0)
        current.append(next_point)
        if current == target:
            return
        time.sleep(time_per_point)

In [31]:
check_stable()

In [33]:
set_field(100, 1000)

In [149]:
read_field()

0

In [120]:
vsm.query("READ?")[18:26].lower()

'ffb93999'

In [121]:
hex_to_num('fdc20b94')

0

In [25]:
set_field(10000, 1000)

In [187]:
def set_and_wait(f, rate, wait_for_stable=5, wait_timeout=600, error=0.5):
    set_field(f, rate)
    stable_count = 0
    i = 0
    while i < wait_timeout and stable_count < wait_for_stable:
        i += 1
        if abs(read_field() - f) < error:
            stable_count += 1
        else:
            stable_count = 0
        time.sleep(1)
    print("current field is: {}G".format(read_field()))

In [60]:
set_and_wait(30, 50, wait_for_stable=5, wait_timeout=600, error=0.5)

current field is: 29.862655639648438G


In [15]:
SENSITIVITIES = [
        0.0, 2.0e-9, 5.0e-9, 10.0e-9, 20.0e-9, 50.0e-9, 100.0e-9,
        200.0e-9, 500.0e-9, 1.0e-6, 2.0e-6, 5.0e-6, 10.0e-6,
        20.0e-6, 50.0e-6, 100.0e-6, 200.0e-6, 500.0e-6, 1.0e-3,
        2.0e-3, 5.0e-3, 10.0e-3, 20.0e-3, 50.0e-3, 100.0e-3,
        200.0e-3, 500.0e-3, 1.0
    ]
TIME_CONSTANTS = [
        10.0e-6, 20.0e-6, 50.0e-6, 100.0e-6, 200.0e-6, 500.0e-6,
        1.0e-3, 2.0e-3, 5.0e-3, 10.0e-3, 20.0e-3, 50.0e-3, 100.0e-3,
        200.0e-3, 500.0e-3, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0,
        100.0, 200.0, 500.0, 1.0e3, 2.0e3, 5.0e3, 10.0e3,
        20.0e3, 50.0e3, 100.0e3
    ]

In [16]:
def init_ac(ac, freq=157, start_I=0.002, stand_by_I=0.002):
    ac.write("SOUR:WAVE:ABOR")
    ac.write("SOUR:WAVE:AMPL {}".format(start_I))
    ac.write("SOUR:WAVE:FREQ {}".format(freq))
    ac.write("SOUR:WAVE:ARM")
    ac.write("SOUR:WAVE:INIT")
    time.sleep(3)
    ac.write("SOUR:WAVE:AMPL {}".format(stand_by_I))
    

In [17]:
def init_ametek(ame, time_constant=0.5, sensitivity=10.0e-6, harmonic=1):
    ame.write("TC {}".format(TIME_CONSTANTS.index(time_constant)))
    ame.write("SEN {}".format(SENSITIVITIES.index(sensitivity)))
    ame.write("REFN {}".format(harmonic))

In [18]:
def lockin_always_on(ame, time_per_point=0.1, points_stored=5, show_data=False, skip=50):
    X = []
    Y = []
    M = []
    P = []
    for _ in range(skip + points_stored):
        time.sleep(time_per_point)
        m, p = ame.query('MP.').replace("\n\x00", "").split(",")
        x, y = ame.query('XY.').replace("\n\x00", "").split(",")
        X.append(float(x))
        Y.append(float(y))
        M.append(float(m))
        P.append(float(p))
    
    res = []
    all_result = [X, Y, M, P]

    for i in all_result:
        i = i[-points_stored:]
        i = np.array(i)
        res.append(i.mean())
        res.append(i.std())
    if show_data:
        print(all_result)
    return res

In [19]:
fileds = [14000]
steps = [1000]

In [20]:
def generate_fields(fileds, steps):
    assert (len(fileds) == len(steps))
    res = []
    for i in range(len(fileds)):
        res.extend([f for f in range(fileds[i], fileds[i + 1] if i+1 < len(fileds) else 0, 
                                   -1 * steps[i])])
    res = res + [0] + [-1* f for f in res[::-1]]
    res += res[::-1][1:]
    return res  

In [21]:
fs = generate_fields(fileds, steps)
print(fs)

[14000, 13000, 12000, 11000, 10000, 9000, 8000, 7000, 6000, 5000, 4000, 3000, 2000, 1000, 0, -1000, -2000, -3000, -4000, -5000, -6000, -7000, -8000, -9000, -10000, -11000, -12000, -13000, -14000, -13000, -12000, -11000, -10000, -9000, -8000, -7000, -6000, -5000, -4000, -3000, -2000, -1000, 0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000]


In [37]:
def field_depend(ame, frequency, current, points_stored, fields, skip=10,
                 ramp_rate=50, time_per_point=0.1, sample_id="test"):
    
    #init_ac(keith, freq=frequency, start_I=current, stand_by_I=current)
    filename = sample_id + "_ac_{}_on_f_{}.csv".format(current, frequency)
    
    with open(filename, 'w') as f:
        f.write("X,dX,Y,dY,M,dM,P,dP,Field\n")
        
    for f in fields:
        print("setting {}".format(f))
        set_field(f, ramp_rate)
        res = lockin_always_on(ame, time_per_point=time_per_point, 
                         points_stored=points_stored, show_data=False, skip=skip)
        res.append(f)
        res = [str(i) for i in res]
        with open(filename, 'a') as file:
            file.write(",".join(res) + "\n")

In [46]:
field_depend(ametek7270, frequency=57, current=0.003, points_stored=10, fields=fs, skip=10,
                 ramp_rate=3000, time_per_point=0.2, sample_id="Fe2O330nm_pt2_10um_57Hz_RT_3mA")

setting 14000


setting 13000


setting 12000


setting 11000


setting 10000


...

setting 10000


setting 11000


setting 12000


setting 13000


setting 14000


In [201]:
set_field(-5000, 50)

In [22]:
def field_depend_gauss(ame, frequency, current, points_stored, fields, skip=10,
                 ramp_rate=50, time_per_point=0.1, sample_id="test", gauss_per_point=0.5, last_field=None):
    
    #init_ac(keith, freq=frequency, start_I=current, stand_by_I=current)
    filename = sample_id + "_ac_{}_on_f_{}.csv".format(current, frequency)
    
    with open(filename, 'w') as f:
        f.write("X,dX,Y,dY,M,dM,P,dP,Field\n")
        
    while True:
        f = next_stable_field(last_field)
        last_field = f
        print("current field: {}".format(f))
        res = lockin_always_on(ame, time_per_point=time_per_point, 
                         points_stored=points_stored, show_data=False, skip=skip)
        res.append(f)
        res = [str(i) for i in res]
        with open(filename, 'a') as file:
            file.write(",".join(res) + "\n")

In [27]:
field_depend_gauss(ametek7270, frequency=57, current=0.003, points_stored=10, fields=fs, skip=10,
                 ramp_rate=3000, time_per_point=0.3, sample_id="Fe2O330nm_pt2_10um_57Hz_RT_5mA_270K_gauss")

current field: 14800


wait field time out
current field: None


current field: 0


current field: 14800


current field: 13800


...

current field: 1000


current field: 0


current field: -1000


current field: -2000


PermissionError: [Errno 13] Permission denied: 'Fe2O330nm_pt2_10um_57Hz_RT_5mA_150K_gauss_ac_0.003_on_f_57.csv'